In [64]:
#Disclaimer ----------------------------------------------------------
#---------------------------------------------------------------------
#This code for learning purpose only 
#it may or may not give you high accuracy required for competition .
#since i dont have GPU to train u used my laptop 
#----------------------------------------------------------------------

from keras.models import Sequential 
from keras.layers.core import Dense,Lambda,Flatten,Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D,ZeroPadding2D
from keras.optimizers import SGD, RMSprop
from keras.utils.data_utils import get_file
from keras.preprocessing import image
import numpy as np

In [70]:
import os
FILES_PATH = 'http://files.fast.ai/models/' #path for vgg network weight . you can download from official website also 
path = "Data/DogVsCat/"
#path = "Data/DogVsCat/sample/"

model_path = path + 'models/' #to save weight for our final model
if not os.path.exists(model_path): os.mkdir(model_path)

In [71]:
def convolution2d(layer,model,filter):
    for i in range(layer):
        model.add(ZeroPadding2D((1,1)))
        model.add(Convolution2D(filter,3,3,activation='relu'))
    model.add(MaxPooling2D((2,2),strides=(2,2)))

In [72]:
def FullyConnected(model):
    model.add(Dense(4096,activation='relu'))
    model.add(Dropout(0.5))

In [73]:
# Mean of each channel as provided by VGG researchers
vgg_mean = np.array([123.68, 116.779, 103.939]).reshape((3,1,1))

def vgg_preprocess(x):
    x = x - vgg_mean 
    # subtract mean
    return x[:, ::-1]    # reverse axis bgr->rgb

In [74]:
def VGG16():
    model = Sequential()
    #First Layer 
    model.add(Lambda(vgg_preprocess,input_shape=(3,224,224)))
    #Block 1 with 2 layer
    convolution2d(2,model,64)
    #Block 2 with 2 layer
    convolution2d(2,model,128)
    #Block 3 with 3 layer 
    convolution2d(3,model,256)
    #Block 4 with 3 layer
    convolution2d(3,model,512)
    #Block 5 with 3 layer
    convolution2d(3,model,512)
    #Flatten Layer
    model.add(Flatten())
    #Fully Connected Layer
    FullyConnected(model)
    FullyConnected(model)
    #output layer with softmax
    model.add(Dense(1000,activation='softmax'))
    return model

In [75]:
model = VGG16() # create our own VGG16 network 

C:\Users\Ali\Anaconda3\lib\site-packages\keras\layers\core.py:622: UserWarning: `output_shape` argument not specified for layer lambda_1 and cannot be automatically inferred with the Theano backend. Defaulting to output shape `(None, 3, 224, 224)` (same as input shape). If the expected output shape is different, specify it via the `output_shape` argument.
  .format(self.name, input_shape))


In [77]:
fpath = get_file('vgg16.h5', FILES_PATH+'vgg16.h5', cache_subdir='models')#download weight for  model 
model.load_weights(fpath)#load weight

In [78]:
model.pop() #pop the last layer and add our own layer , since this classifier doesnt have 1000 classes  

In [79]:
for layer in model.layers:
    layer.trainable=False #do not train first 15 layer 

model.add(Dense(2,activation='softmax')) # add one Fully connected layer  with 2 output because we have 
                                        #classes and train only this layer  

model.compile(optimizer=RMSprop(lr=0.1), loss='categorical_crossentropy', metrics=['accuracy'])#set optimizer 

In [11]:
batch_size=64
def get_batches(dirname, gen=image.ImageDataGenerator(), shuffle=True, 
                batch_size=batch_size, class_mode='categorical'):
    return gen.flow_from_directory(path+dirname, target_size=(224,224), 
                class_mode=class_mode, shuffle=shuffle, batch_size=batch_size)

In [12]:
train_images = get_batches('train',batch_size=batch_size)
val_images = get_batches('val',batch_size=batch_size)
img,label = next(val_images)

Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


In [15]:
model.fit_generator(train_images, samples_per_epoch=train_images.n, nb_epoch=1, validation_data=val_images, nb_val_samples=val_images.n)

#you can set your own epoch size . 
#i only train it for one epoch because of resource limitation (only one epoch took me  5 hours to train)
           

Epoch 1/1
20000/20000 [==============================] - 15473s - loss: 0.1057 - acc: 0.9629 - val_loss: 0.0950 - val_acc: 0.9714


In [19]:
model.save_weights(model_path+'finetune1.h5')
model.load_weights(model_path+'finetune1.h5')
# save all trained weight , so you can use it later

done


In [12]:
test_image = get_batches('test',batch_size=batch_size) # load test image 

Found 12500 images belonging to 1 classes.


In [13]:
results = model.predict_generator(test_image,test_image.nb_sample)
# predict for all test image

done


In [2]:
# bcolz you to save array in your directory 
import bcolz
def save_array(fname, arr):
    c=bcolz.carray(arr, rootdir=fname, mode='w')
    c.flush()


def load_array(fname):
    return bcolz.open(fname)[:]

In [20]:
#saving predicted result in local directory
save_array(path + 'test_preds.dat', result)
save_array(path + 'filenames.dat', test_image.filenames)

In [5]:
result = load_array(path+'test_preds.dat')
filename = load_array(path+'filenames.dat')

In [7]:
result.shape

(12500, 2)

In [8]:
result[:10]

array([[  1.34522736e-03,   9.98654783e-01],
       [  3.51411253e-01,   6.48588777e-01],
       [  4.68529016e-03,   9.95314717e-01],
       [  1.00000000e+00,   2.16642958e-12],
       [  1.47995213e-02,   9.85200465e-01],
       [  9.99998391e-01,   1.57970237e-06],
       [  9.25120115e-02,   9.07487988e-01],
       [  1.00000000e+00,   2.34677746e-14],
       [  1.00000000e+00,   1.53770524e-13],
       [  1.71022187e-03,   9.98289764e-01]], dtype=float32)

In [31]:
#get ids for your prediction . for submitting in to competition
ids = np.array([int(f[5:f.find('.')]) for f in filename])

In [30]:
filename[:5]

array(['test\\1.jpg', 'test\\10.jpg', 'test\\100.jpg', 'test\\1000.jpg',
       'test\\10000.jpg'],
      dtype='<U14')

In [32]:
ids[:5]

array([    1,    10,   100,  1000, 10000])

In [ ]:
is_dog = mresult[:,1]
is_dog = is_dog.clip(0.05,0.95)

In [ ]:
#concat two array 
final_result = np.stack([ids,is_dog],axis=1)

In [62]:
final_result[:5]

array([[  1.00000000e+00,   5.00000007e-02],
       [  1.00000000e+01,   3.51411253e-01],
       [  1.00000000e+02,   5.00000007e-02],
       [  1.00000000e+03,   9.49999988e-01],
       [  1.00000000e+04,   5.00000007e-02]])

In [63]:
#save file as csv for submission
submission_file_name = 'submission.csv'
np.savetxt(submission_file_name, final_result, fmt='%d,%.5f', header='id,label', comments='')